<a href="https://colab.research.google.com/github/good0neighbour/ESTCampAI/blob/main/%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C/ex05_%EC%9D%B4%EB%AF%B8%EC%A7%80_%EA%B8%B0%EB%B0%98_%EC%9D%8C%EC%95%85_%EC%B6%94%EC%B2%9C_%EA%B8%B0%EB%8A%A5_%EA%B5%AC%ED%98%84_CLIP_Multimodal_Embedding(%EB%A9%80%ED%8B%B0%EB%AA%A8%EB%8B%AC_%EC%9E%84%EB%B2%A0%EB%94%A9_%EA%B8%B0%EB%B0%98_%EC%BD%98%ED%85%90%EC%B8%A0_%ED%95%84%ED%84%B0%EB%A7%81).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. 기존 이미지 분류(Classification)의 작동 방식
- 이미지 -> 하나의 정답 라벨(label) 예측하는 모델
  - ex) 개 / 고양이 분류
- 대량의 이미지 + 라벨 필요
- 모델은 라벨을 맞히는 방향으로 학습
- 출력 : [개 : 0.9, 고양이 0.1]
**한계점**
- 새로운 클래스 추가 시 -> 재학습 필요
- 라벨 수 증가 -> 데이터, 비용 증가
- 자연어와 연결 불가
  - '평화로운 사진인가요?' 같은 질문이 불가능

### 2. CLIP : 이미지와 텍스트를 같은 방식으로 다루기
**기본 아이디어**
> 이미지와 텍스트를 같은 벡테 공간(embedding space)에 매핑한다.
> 이미지 -> 벡터, 텍스트 -> 벡터
> 새로운 라벨 필요 없음
> 라벨에 대한 확률이 아니라 -> 512차원 임베딩 벡터 출력
> 인터넷 이미지 + 캡션 4억 쌍으로 학습
> "이미지-문장" 쌍을 기반으로 의미를 학습

### 3. 이미지 분류, CLIP 비교
- 기존 이미지 분류 : 시험을 보는 학생
  - 시험지가 항상 정해져 있음 '개/고양이만 나옴'
  - 새로운 질문에는 답할 수 없음
- CLIP : 이미지를 문장처럼 이해하는 학생
  - 이미지의 분위기, 느낌을 이해함
  - "여름 느낌인가?", "평화로운가?", "여행지인가?", "이 이미지와 어울리는 음악은"

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/ESTCampAI/추천시스템

/content/drive/MyDrive/ESTCampAI/추천시스템


In [3]:
import pandas as pd
df1 = pd.read_csv('data/music.csv')
df2 = pd.read_csv('data/places.csv')
display(df1.head(3))
display(df2.head(3))

,가수,제목,가사,장르
0,SG워너비,Timeless,어쩜 살아가다 보면 한 번은 날 찾을지 몰라 난 그 기대 하나로 오늘도 힘겹게 버틴...,발라드
1,성시경,희재,햇살은 우릴 위해 내리고 바람도 서롤 감싸게 했죠 우리 웃음속에 계절은 오고 또 갔...,발라드
2,SG워너비,라라라,그대는 참 아름다워요 밤 하늘의 별빛보다 빛나요 지친 나의 마음을 따뜻하게 감싸줄 ...,발라드


,이름,카테고리,설명,이미지URL
0,목포해상케이블카,즐길거리,"국내 최장 3.23km, 목포해상케이블카는 목포 시내 북항스테이션을 출발하여 유달산...",http://www.mmcablecar.com/images/sub/cablecar/...
1,목포김대중노벨평화상기념관,명소,김대중노벨평화상 기념관은 김대중 전 대통령의 생애를 통해 민주주의·인권·평화의 의미...,http://kdjnpmemorial.or.kr/data/content/1_1_h
2,목포오거리식당,먹을거리,국무 총리께서 즐겨드시던 밥상으로 코스요리가 구성된 곳,https://search.pstatic.net/common/?src=https%3...


In [4]:
!pip install requests

In [5]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-agnvf3w5
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-agnvf3w5
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.5 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=717fbeb8d704cca47349c106b55fe4c399afd917b6aebd70eb69781b5cb9abab
  Stored in directory: /tmp/pip-ephem-wheel-cache-g6sricg4/wheels/35/3e/df/3d24cbfb3b6a06f17a2bfd7d1138900d4365d9028aa8f6e92f
Successfully built clip


In [6]:
import csv
import requests
import os
import urllib3

# 경고창 끄기(verify = False 쓸 때 경고 전역으로 끄기)
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# 이미지 저장 폴더 생성(이미 있는 경우 X)
os.makedirs('data/images', exist_ok=True)

with open('data/places.csv', 'r', encoding='utf-8',newline='') as f:
    reader = csv.reader(f) # row(행) 하나씩 리스트로 읽어옴
    next(reader) # 헤더 한 줄 스킵

    for row in reader:
        url = row[3]
        if not url:
            continue

        print(row)

        # 이미지 url로 http get 요청
        # vertify = False .png, .jpg 같은 확장자 형태의 url은 크게 문제가 없지만
        # 확장자 형태가 아닌 url에서 ssl 인증서 오류가 발생할 때가 있음
        # ssl 검증을 끄고 요청하겠다
        resp = requests.get(url, timeout=10, verify=False)

        with open(f'data/images/{row[0]}.jpg', 'wb') as img:
            img.write(resp.content)


['목포해상케이블카', '즐길거리', '국내 최장 3.23km, 목포해상케이블카는 목포 시내 북항스테이션을 출발하여 유달산 정상부에서 ‘ㄱ’자로 꺾여, 해상을 지나 반달섬 고하도에 이르는 국내 최장 케이블카입니다.', 'http://www.mmcablecar.com/images/sub/cablecar/home1-2.jpg']
['목포김대중노벨평화상기념관', '명소', '김대중노벨평화상 기념관은 김대중 전 대통령의 생애를 통해 민주주의·인권·평화의 의미와 가치를 전하며 한국인 최초로 노벨평화상을 수상하기까지의 모습을 통해 자라나는 세대들에게 객관적 사실과 사료에 의거해 체험적 역사교육의 장', 'http://kdjnpmemorial.or.kr/data/content/1_1_h']
['목포오거리식당', '먹을거리', '국무 총리께서 즐겨드시던 밥상으로 코스요리가 구성된 곳', 'https://search.pstatic.net/common/?src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20200107_236%2F1578386002086aXjq3_JPEG%2FFnxGHkoYlUtBG3dcF9Cud_U9.jpeg.jpg']
['나주  국립나주박물관', '즐길거리', '영산강 유역에 남아있는 고고자료를 전시', 'https://lh3.googleusercontent.com/p/AF1QipOyT6V14Oq3CP2wWN_H_XgN9HpV6Eoc6dok3xyM=s680-w680-h510']
['나주  한국천연염색박물관', '즐길거리', '전국에서 유일하게 국가중요무형문화재 제 115호 염색장이 활동하고 있는 곳', 'https://lh3.googleusercontent.com/p/AF1QipNNnV8__2OVpElXoaw-aE0wnNUkpMi_0itElYF3=s680-w680-h510']
['나주  영상테마파크', '즐길거리', '드라마 영화 촬영장이며 고구려의 건국 역사와 엔터테인먼트가 결합된 영상전문 테마공원', 'https://lh3.goo

In [7]:
import os
import csv
import torch
import clip
from PIL import Image
import json
from tqdm.auto import tqdm

DEVICE = "cpu"
# 사용할 디바이스 설정
# - GPU 사용 가능하면 "cuda"로 바꾸면 됨
MODEL_NAME = "ViT-B/32"
# 사용할 CLIP 모델 이름(ResNet기반 : RN50, RN101 등등, ViT기반 :ViT-B/16, ViT-L/14)

model, preprocess = clip.load(MODEL_NAME, device=DEVICE)
# CLIP 모델, 전처리 함수 로드
# model : 이미지/텍스트를 512차원 임베딩 벡터로 변환해주는 신경망
# preprocess : 일반 이미지(PIL Image)를 CLIP이 요구하는 텐서 형태로 변환해주는 함수
# numpy ndarray -> cpu기반 연산 / tensor -> gpu를 사용한 연산 가능

def load_place_descriptions(csv_path="data/places.csv"):
  # CLIP은 이미지+컨셉으로 학습된 모델
  # places.csv 파일을 읽어서 "이미지명 - 장소 설명 텍스트" 형태의 dict를 만드는 함수

    mapping = {}

    with open(csv_path, "r", encoding="utf-8") as f:
        reader = csv.reader(f)
        next(reader)

        for row in reader:
          # CSV : [이름, 카테고리, 설명, URL]
            name = row[0] # 장소 이름
            desc = row[2] # 장소 설명 텍스트

            filename = f"{name}.jpg"

            # dict 저장
            mapping[filename] = desc

    return mapping


def extract_place_features(image_weight: float = 0.7, text_weight: float = 0.3):
    # 텍스트가 더 중요한 경우(ex. 검색) : 텍스트 가중치를 높여볼 수 있다.
    # 1. 이미지 임베딩(CLIP Img Encoder)
    # 2. 텍스트 임베딩(CLIP txt Encoder)
    # 3. 두 벡터의 가중합을 계산하여 최종 벡터를 저장

    features = {}
    images_dir = os.path.join(os.getcwd(), "data/images")

    desc_map = load_place_descriptions("data/places.csv")
    # "이름 : 설명" 매핑값 가져오기

    for image_name in tqdm(os.listdir(images_dir)):
      # 구글 drive 혹은 local 지정 폴더에서 파일의 목록 가져와서 사용하기
        image_path = os.path.join(images_dir, image_name)

        # 예외처리
        # 이미지 로드 + 전처리
        try:
            image = preprocess(Image.open(image_path)).unsqueeze(0).to(DEVICE)
            # Image.open : 파일에서 이미지 로드
            # preprocess : 텐서로 변환을 해줌
            # unsqueeze(0) : 차원을 하나 추가해주는 함수
            # 이미지 : (행 픽셀, 열 픽셀, 색상)
            # to(DEVICE) : cpu 혹은 gpu로 텐서를 옮김

        except Exception:
            # 이미지 파일이 깨졌거나 열 수 없는 경우
            print(f'Error: {image_name}')
            continue

        with torch.no_grad():
          # 임베딩 -> 예측, 추론을 해서 뽑아내는 값
          # gradient(기울기)를 계산하지 않게 하여 연산량, 메모리 사용량을 줄여줌
          # 임베딩 모델은 보통 "학습"이 아니라 "피쳐 추출"만 하기 때문에 사용
            img_feat = model.encode_image(image)
            # 512 차원의 벡터로 이미지를 변경해줌

        desc = desc_map.get(image_name, None)
        # 이미지에 해당하는 장소 설명 텍스트 가져오기

        if desc:
            text_tokens = clip.tokenize([desc]).to(DEVICE)
            # 텍스트 토큰화
            # clip.tokenize : 문장을 CLIP이 이해할 수 있는 토큰 텐서로 변환

            with torch.no_grad():
                txt_feat = model.encode_text(text_tokens)
                # 512차원의 임베딩으로 텍스트를 변환

            img_norm = img_feat / img_feat.norm(dim=-1, keepdim=True)
            txt_norm = txt_feat / txt_feat.norm(dim=-1, keepdim=True)
            # 벡터 / 길이로 나누어 길이가 1인 단위 벡터로 만든다.
            # L2 정규화를 진행해야 벡터를 단위 벡터로 만들 수 있고
            # 두 벡터의 내적(dot product)이 곧 코사인 유사도가 된다.
            # CLIP 논문에서도 이미지/텍스트 임베딩을 위와 같은 구조로 진행하였음

            fused = image_weight * img_norm + text_weight * txt_norm
            fused = fused / fused.norm(dim=-1, keepdim=True)

            final_feat = fused
            # 최종적으로 사용할 장소 임베딩 벡터

        else:
            img_norm = img_feat / img_feat.norm(dim=-1, keepdim=True)
            final_feat = img_norm

        features[image_name] = final_feat.cpu().numpy().tolist()
        # tensor -> numpy -> list로 변환

    with open("place_features.json", "w", encoding='utf-8') as fp:
        json.dump(features, fp, ensure_ascii=False)
        # ensure_axcii = False : 한글이 깨지지 않도록 설정


def extract_music_features():
  # music.csv 파일을 읽어서 '가사' 컬럼을 기준으로 CLIP 텍스트 임베딩을 계산하여
  # json에 저장하는 함수

    features = {}

    file = open('data/music.csv', 'r', encoding='utf-8')
    # music.csv 파일 열기(단순 문자열 처리 방식)

    for row in tqdm(file):
        if '"' in row:
            lyrics = row.split('"')[1]
            row = row.replace(f'"{lyrics}"', lyrics.replace(',', ' '))

        try:
            artist, title, lyrics, genre = row.split(',')
            # 가수, 제목, 가사, 장르 순으로 분리
        except Exception:
            print(f'Error: {row}')
            continue

        for i in range(0, len(lyrics), 40):
            text = clip.tokenize([lyrics[i:i+40]]).to(DEVICE)
          # 긴 가사를 40글자 단위로 잘라서 여러 조각에 대해 임베딩을 만들어줌
          # 잘라낸 가사를 CLIP 토큰화

          # 성능을 올리기위한 방안
          # 띄어쓰기, 불용어처리, 특수문자 제거 등의 클렌징 작업
          # okt 등의 형태소분석기를 사용하여 원하는 품사만 걸러내는 작업 후 임베딩을 진행하면
          # 더 좋은 성능을 기대해 볼 수도 있다.

            embeddings = []

            with torch.no_grad():
                text_features = model.encode_text(text)
                embeddings.append(text_features.cpu().numpy().tolist())

            text_features = torch.tensor(embeddings).mean(dim=0)
            # 40글자 단위로 조각을 내었기 때문에 여러 조각의 평균을 구해줌
            name = f'{artist}-{title}'.replace(' ', '_')

            features[name] = text_features.cpu().numpy().tolist()

    with open("data/music_features.json", "w", encoding='utf-8') as fp:
        json.dump(features, fp, ensure_ascii=False)


def chech_feature_counts():
    # json 파일 안에 어떤 곡명이 들어있는지, 총 몇곡인지 출력해주는 디버깅 함수
    with open("music_features.json", "r", encoding='utf-8') as fp:
        features = json.load(fp)

    for key, value in features.items():
        print(key)

    print(len(features))

if __name__ == "__main__":
  # 이 파이썬 파일을 "직접 실행"할 때만 아래 코드가 동작한다.
  # __name__ : 현재 모듈의 이름을 담고 있는 내장 변수
  # __main__ : 직접 실행된 모듈의 경우 __name__ 값을 가짐
  # 다른 파일에서 import 해서 사용하는 경우 모듈의 이름 값을 가짐
    extract_place_features()
    extract_music_features()

    # 디버깅용
    # chech_feature_counts()


100%|████████████████████████████████████████| 338M/338M [00:01<00:00, 247MiB/s]


  0%|          | 0/61 [00:00<?, ?it/s]

Error: 목포김대중노벨평화상기념관.jpg
Error: 나주  황포 돛배 .jpg
Error: 곡성 섬진강 문화학교 캠핑장.jpg


0it [00:00, ?it/s]

In [8]:
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


def recommend_music(place_name: str, top_k: int = 5):

    with open("data/place_features.json", "r", encoding="utf-8") as fp:
        place_features = json.load(fp)

    with open("data/`music_features.json", "r", encoding="utf-8") as fp:
        music_features = json.load(fp)

    place_feature = (
        np.array(place_features[place_name], dtype=float)
        # 저장해둔 json은 dict 형태이기 때문에 ndarray로 형변환
        .flatten()
        .reshape(1, -1)
    )

    similarities = {}
    # "노래이름 : 유사도 값" 형태로 저장하기 위한 dict

    for music_name, music_feature in music_features.items():
        music_vec = (
            np.array(music_feature, dtype=float)
            .flatten()
            .reshape(1, -1)
        )

        sim = cosine_similarity(place_feature, music_vec)[0, 0]

        similarities[music_name] = float(sim)

    similarities = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
    # sorted -> similarities는 dict기 때문에 sort_values 사용 불가
    # key(정렬 기준) -> lambda x:x[1] 자료 중에서 첫 번째 인덱스(유사도) 값을 기준으로 정렬하겠다

    return similarities[:top_k]


if __name__ == "__main__":
    print(recommend_music("여수 아쿠아플라넷.jpg"))
    print(place_features)

[('아이브-_러브다이브', 0.22915844287933168), ('임창정-소주_한_잔', 0.22173145195139732), ('뉴진스-_gods', 0.21424108182440715), ('영탁-막걸리_한잔', 0.21178958651914967), ('영탁-찐이야', 0.2102723382045015)]


In [8]:
with open("data/place_features.json", "r", encoding="utf-8") as fp:
        place_features = json.load(fp)

place_features.keys()
# 유사도를 높이기 위한 방안
# 현재 음악데이터 101ea, 여행지 데이터 61ea로 데이터의 양이 매우 적음
# 1. 데이터의 개수를 늘리는 방안
# - 여행지의 개수 자체를 늘리는 방안
# - 한 개의 여행지에 여러 사진을 사용하는 방법(설명은 동일하게 유지하기)
# - 상품 추천(옷) : 옷에 대한 여러방향의 사진, 조도, 명암에 따른 사진 여러장 + 옷에 대한 설명(고정)
# 2. 콘텐츠 기반 필터링에서 진행한 것과 다르게 음악 데이터에 별도의 전처리 X
# - 데이터 클렌징, 형태소 분석 등의 다양한 전처리를 통해서 성능 개선 여지 있음
# 3. 이미지를 더 잘 설명할 수 있는 텍스트로 넣어보는 방안